# Data Preparation

Pada bagian ini kita akan melakukan proses pra pengolahan data. Beberapa hal yang biasanya dilakukan adalah,

1. Load data
2. Cek _missing value_
3. Impute _missing value_ jika diperlukan
4. Cek data kategorikal / numerikal
5. Manipulasi kolom (jika diperlukan)

In [3]:
# import library yang dibutuhkan
import numpy as np
import pandas as pd

In [4]:
# Load data

df = pd.read_csv('home_data.csv')

# Display 5 data teratas
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [ ]:
# Cek dimensi data -> Jumlah baris dan kolom
print(f'Dimensi: {df.shape}')
print(f'Jumlah Baris: {df.shape[0]}')
print(f'Jumlah Kolom: {df.shape[1]}')

In [ ]:
# Cek nama-nama kolom yang ada didalam data
df.columns

In [ ]:
# Cek informasi umum tentang data
df.info()

In [ ]:
# Cek nilai statistik deskriptif
df.describe()

In [ ]:
# Cek jumlah data pada kolom dengan nilai kosong / NA
df.isnull().sum()

In [ ]:
from datetime import datetime

# Manipulasi kolom -> date
df["date_converted"] = pd.to_datetime(df["date"]) # waktu pencatatan data

# Buat kolom baru untuk umur rumah
current_year = datetime.now().year
df["age_of_house"] = current_year - pd.to_datetime(df["date"]).dt.year

# Buang kolom date
df = df.drop(columns=["date"], axis=1)


df.head()

In [ ]:
# Visualisasi Distrubusi Harga Rumah, Luas Bangunan, dan Luas Tanah, dan Usia Bangunan
# Misal kita tertarik untuk lebih lanjut mengetahui sebaran datanya

# Plot Histogram
df.hist(column=["price", "sqft_living", "sqft_lot", "age_of_house"], bins=20, figsize=(12,3), layout=(1,4))

In [ ]:
# Plot Boxplot
# Dapat membantu kita untuk menjukkan nilai nilai yang outlier
df.boxplot(column=["price", "sqft_living", "sqft_lot", "age_of_house"], grid=False, figsize=(5,5))

In [ ]:
# Cek korelasi antar variabel
# Pada konteks regresi, hal ini sangat sensitif terhadap hasil prediksi.
# Idealnya, hubungan yang kuat harus terjadi antara independen variabel dengan dependen variabel.
# Hubungan yang kuat antar indenpen variabel dapat menjadi masalah --> MULTICOLINEARITY

import matplotlib.pyplot as plt
import seaborn as sns

corr = df.corr().round(2)
plt.figure(figsize=(13,10))
sns.heatmap(corr, annot = True, cmap = 'Blues')

In [ ]:
# Cek hubungan antara Dependent Variabel (Target) dengan Independent Variabel (Fitur)
# dengan menggunakan scatter plot
# Target = Price
# Fitur = variabel-variabel yang lain

sns.pairplot(df)

In [ ]:
# Cukup sulit mendapatkan informasi jika kita melakukan scatter plotting untuk semua kolom
# Maka, nilai korelasi akan sangat membantu kita dalam mengetahui hubungan antara target-fitur, fitur-fitur
# Sebagai contoh, kita cek saja hubungan antara price dengan fitur yang memiliki nilai korelasi > 0.5

# Cek Nilai Korelasi
high_corr = corr["price"]
print(high_corr)

In [ ]:
# Plot Korekasi > 0.5
hi_corr_features = df[["bathrooms", "sqft_living", "grade", "sqft_above", "sqft_living15"]]
target = df[["price"]]

sns.pairplot(df,x_vars=hi_corr_features,y_vars=target)

# Feature Selection

Berdasarkan tahap sebelumnya, kita mengetahui hanya ada beberapa fitur / independend variabel yang berpengaruh terhadap target (dependend variabel). Oleh karena itu, kita hanya akan menggunakan data tersebut.

In [ ]:
# Pilih fitur dengan korelasi > 0.5

X = df[["bathrooms", "sqft_living", "grade", "sqft_above", "sqft_living15"]] # fitur
y = df[["price"]] # target

## Split data

Untuk kebutuhan pembuatan model (machine learning). Kita akan split data menjadi data training dan data testing

In [ ]:
# import package
from sklearn.model_selection import train_test_split

# split data menjadi 70% training dan 30% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Model

Kasus yang akan kita pecahkan adalah bagaimana cara menentukan harga rumah dengan kualifiasi tertentu. Target merupakan nilai riil dari harga rumah. Sehingga, kita dapat menggunakan model regresi untuk memprediksi harga rumah. Salah satu model regresi paling sederhana untuk memecahkan masalah ini adalah __Regresi Linier__

In [ ]:
# import package
from sklearn.linear_model import LinearRegression

# Inisiasi obyek model
lr = LinearRegression()

# Latih model dengan data training (fitting)
lr.fit(X_train, y_train)

# Prediksi harga dengan data testing
y_pred = lr.predict(X_test)

# Evaluasi

Untuk mengetahui performa model yang kita buat, maka kita perlu melakukan evaluasi terdapa model tersebut. Pada kasus regresi, jika hasil prediksi jaraknya semakin jauh dengan nilai sebenernya, maka hasil model kita kurang baik. Pengukuran evaluasi pada model regresi diantaranya dapat menggunakan nilai $R^2$, __Mean Squared Error (MSE)__, __Mean Absolute Error (MAE)__, atau __Root Mean Square Error (RMSE)__.

In [ ]:
# import package
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error

r2_score = r2_score(y_test, y_pred)
mae_score = mean_absolute_error(y_test, y_pred)
mse_score = mean_squared_error(y_test, y_pred)
rmse_score = root_mean_squared_error(y_test, y_pred)

print(f'Skor R2: {r2_score}')
print(f'Skor MAE: {mae_score}')
print(f'Skor MSE: {mse_score}')
print(f'Skor RMSE: {rmse_score}')

## Hasil Evaluasi

Ternyata hasilnya tidak cukup oke. Terbukti dari nilai $R^2$ yang berada pada angka 0.53. Jika model yang kita buat baik, maka nilai $R^2$ akan mendekati 1. Mari kita coba menggunakan semua fitur kecuali id, tanggal-tanggal, lat, long, dan zipcode

## Simulasi Prediksi

In [ ]:
# Buat data baru untuk simulasi prediksi
new_data = pd.DataFrame({
    "bathrooms": [2.0],
    "sqft_living": [1500],
    "grade": [7],
    "sqft_above": [1500],
    "sqft_living15": [1600]
})

# Lakukan prediksi dengan model lr
predicted_price = lr.predict(new_data)

print(f'Prediksi harga rumah: {predicted_price[0]}')

## Buat Model Ulang

In [ ]:
# Drop kolom
df.drop(columns = ['id', 'waterfront', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'date_converted'], axis = 1, inplace = True)

df.head()

In [ ]:
# Cek Kolom
df.columns

In [ ]:
# Pisahkan fitur dan target
# Kali ini kita akan menggunakan iloc

X2 = df.iloc[:,1:] # semua baris, kolom dari indesk ke-1 hingga semua
y2 = df.iloc[:,0] # semua baris, kolom indeks ke-0

In [ ]:
# Split training dan testing

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state=42)

In [ ]:
# Inisiasi obyek model
lr2 = LinearRegression()

# Latih model dengan data training (fitting)
lr2.fit(X2_train, y2_train)

# Prediksi harga dengan data testing
y2_pred = lr2.predict(X2_test)

In [ ]:
# Evaluasi model baru
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error

r2_score2 = r2_score(y2_test, y2_pred)
mae_score2 = mean_absolute_error(y2_test, y2_pred)
mse_score2 = mean_squared_error(y2_test, y2_pred)
rmse_score2 = root_mean_squared_error(y2_test, y2_pred)

print(f'Skor R2: {r2_score2}')
print(f'Skor MAE: {mae_score2}')
print(f'Skor MSE: {mse_score2}')
print(f'Skor RMSE: {rmse_score2}')

## Evaluasi Ulang

Ternyata terjadi peningkatan pada metric $R^2$ mesikpun tidak signifikan, yaitu menjadi 0.58. Proses seperti ini akan terus belanjut hingga kita mendapatkan model yang sesuai dengan harapan.

## Simulasi Prediksi (Ulang)

In [ ]:
# Buat data baru untuk simulasi prediksi
new_data = pd.DataFrame({
    'bedrooms': [3],
    'bathrooms': [2.0],
    'sqft_living': [1500],
    'sqft_lot': [5000],
    'floors': [1.0],
    'view': [0],
    'condition': [3],
    'grade': [7],
    'sqft_above': [1500],
    'sqft_basement': [0],
    'sqft_living15': [1600],
    'sqft_lot15': [5000],
    'age_of_house': [10]
})

# Lakukan prediksi dengan model baru yaitu lr2
predicted_price = lr2.predict(new_data)

print(f'Prediksi harga rumah: {predicted_price[0]}')